In [1]:
#########################################################################################################################
#               Python notebook to automatically read TC team database and format them for MAGEMin
#------------------------------------------------------------------------------------------------------------------------
# The goal here is to take into account any changes of the database with no(ish) effort
# Note that compositional variables with upper cases pose problems to sympy (not sure why...)
# to fix that, they are turned into lower cases compositional variables
# *** a temporary set of functions correct lacking multiply signs from the TC output
# *** some older solution phase formulation use parenthesis in their definition a regular exp. function is used to correct this
#
# last update 05.04.23, N.Riel
#########################################################################################################################

import re
import numpy as np
import math as math
import cmath
import time
import random
from sympy import *
from sympy.printing import print_ccode

# EXPORT
import os, re
init_printing()

In [2]:
# endmember fraction to compositional variables - metabasite database
def get_cv_mp():
    cv_mp = {}

    liq = ['q   = p0',
    'an  = p3',
    'h2o = p7',
    'fsp = p1 + p2',
    'na  = p1/fsp',
    'ol  = 1.0 - q - fsp - an - h2o - p4',
    'xc   = p6/ol']
        
    inout = {'liq':liq}
    cv_mp.update(inout)
    
    pl4tr = [   'k   = p2',
    'ca  = p1']
        
    inout = {'pl4tr':pl4tr}
    cv_mp.update(inout)
 
    bi = [    
    'm  = p6',
    'f  = p5',
    't  = p4',
    'y  = p3',
    'q  = 3.0*(-f*p1 + f - 3.0*m*p1 + m + p0 - p1*t - p1*y + p1 + t + y - 1.0)/(f + 3.0*m + t + y - 3.0)',
    'xc  = (-p2 + q)/(f + 3.0*m + t + y)']
        
    inout = {'bi':bi}
    cv_mp.update(inout)
 
    g = [ 'f   =  p4',
     'z   =  p3',
     'm   =  p2',
     'xc  =  p1/(1.0 - m - z)']
        
    inout = {'g':g}
    cv_mp.update(inout)
 
    ep = [ 'q   =  p1/2.0',
    'f   =  p2 + q' ]
        
    inout = {'ep':ep}
    cv_mp.update(inout)
 
    ma = [    
    'f  = p5',
     'c  = p4',
     'n  = p3',
     'y  = p0 + c + n + f',
     'xc = p2/(1.0-y)']

    inout = {'ma':ma}
    cv_mp.update(inout)
     
 
    mu = [    
    'f  = p5',
     'c  = p4',
     'n  = p3',
     'y  = p0 + c + n + f',
     'xc = p2/(1.0-y)']

    inout = {'mu':mu}
    cv_mp.update(inout)
         
    opx = [ 'c  = p6',
    'm  = p5',
    'f  = p4',
    'y  = p3',
    'xc = (-2.0*p1 - p2)/(c + f + 2.0*m + y - 2.0)',
    'q  = 2.0*(pow(c, 2) + 2.0*c*f + 3.0*c*m + c*p0 + 2.0*c*p1 + c*p2 + 2.0*c*y - 3.0*c + pow(f, 2) + 3.0*f*m + f*p0 + 2.0*f*y - 3.0*f + 2.0*pow(m, 2) + 2.0*m*p0 + 2.0*m*p1 + m*p2 + 3.0*m*y - 4.0*m + p0*y - 2.0*p0 - 2.0*p1 - p2 + pow(y, 2) - 3.0*y + 2.0)/(pow(c, 2) + c*f + 3.0*c*m + c*y - 3.0*c + f*m - f + 2.0*pow(m, 2) + m*y - 4.0*m - y + 2.0)']

    inout = {'opx':opx}
    cv_mp.update(inout)
         
    sa = [ 'f  = p4',
    'y  = p1',
    'xc = (4.0*f + 4.0*p0 + p3 + 4.0*y - 4.0)/(f + y - 4.0)',
    'q  = 1.3333333333333333*(-4.0*pow(f, 2) - 4.0*f*p0 - f*p2 - f*p3 - 8.0*f*y + 8.0*f - 4.0*p0*y + 4.0*p0 - p2*y + 4.0*p2 - p3*y + p3 - 4.0*pow(y, 2) + 8.0*y - 4.0)/(f + y - 4.0)']
    
    inout = {'sa':sa}
    cv_mp.update(inout)
         
    cd = [    'm  = p3',
    'h  = p2',
    'xc = p1/(1.0 - m)']
    
    inout = {'cd':cd}
    cv_mp.update(inout)
    
    st = [    
    't  = p4/(4.0/3.0)',
    'f  = p3',
    'm  = p2',
    'xc = p1/(1.0 - m)'] 
    
    inout = {'st':st}
    cv_mp.update(inout)
    
    chl = ['f   =  p6',
    'm   =  p7',
    'qal = (m + 1.0 - f -(p1 - p3+p5-p0 + p2+p4))/4.0',
    'y   =  p2 + qal',
    'xc  = (-2.0*qal + m + p0 - 4.0*p3 - 5.0*p4 - p5)/(f + 5.0*m + 2.0*y - 6.0)',
    'q4  = (-2.0*pow(qal,2.0) - 2.0*qal*f + qal*m + qal*p0 - 4.0*qal*p3 - 5.0*qal*p4 - qal*p5 - 2.0*qal*y + 2.0*qal + f*m + f*p0 - 4.0*f*p3 - 4.0*f*p4 - f*p5 + 5.0*m*p4 + m*y - m + p0*y - p0 - 4.0*p3*y + 4.0*p3 - 3.0*p4*y - p4 - p5*y + p5)/(qal*f + 5.0*qal*m + 2.0*qal*y - 6.0*qal + pow(f,2.0) + 5.0*f*m + 3.0*f*y - 7.0*f + 5.0*m*y - 5.0*m + 2.0*pow(y,2.0) - 8.0*y + 6.0)',
    'q1  = (10.0*pow(qal,2.0) - 2.0*qal*f - 25.0*qal*m - 5.0*qal*p0 + 20.0*qal*p3 + 25.0*qal*p4 + 5.0*qal*p5 - 14.0*qal*y + 22.0*qal - 4.0*pow(f,2.0) - 21.0*f*m - f*p0 - 4.0*f*p1 + 4.0*f*p3 - 4.0*f*p4 + f*p5 - 12.0*f*y + 28.0*f - 20.0*m*p1 - 45.0*m*p4 - 17.0*m*y + 21.0*m + 3.0*p0*y + p0 - 8.0*p1*y + 24.0*p1 - 12.0*p3*y - 4.0*p3 - 33.0*p4*y + 49.0*p4 - 3.0*p5*y - p5 - 8.0*pow(y,2.0) + 32.0*y - 24.0)/(5.0*(-qal*f - 5.0*qal*m - 2.0*qal*y + 6.0*qal + f*m + f*y - f + 5.0*pow(m,2.0) + 7.0*m*y - 11.0*m + 2.0*pow(y,2.0) - 8.0*y + 6.0))']
    
    inout = {'chl':chl}
    cv_mp.update(inout)
    
    ctd = ['f  = p3',
    'm  = p2',
    'xc = p1/(1.0 - m)']

    inout = {'ctd':ctd}
    cv_mp.update(inout)
    
    sp = [    
    'z  = p3',
    'y  = 1.0 - p2 - z',
    'xc = (-p1 + z + 1.0)/(z + 1.0)']
    
    inout = {'sp':sp}
    cv_mp.update(inout)
    
    ilm = [   'm  = p4',
    'z  = p3',
    'q  = p0',
    'xc  = 1.0 - p2']
    
    inout = {'ilm':ilm}
    cv_mp.update(inout)
    
    mt = [   'xc  = 1.0 - p2',
    'q   = (3.0*xc - p1)/3.0']
    
    inout = {'mt':mt}
    cv_mp.update(inout)
    
    return cv_mp

def get_cv_mb():
    cv_mb = {}

    L = [   'yan = p8',
            'na  = p1/(p1 + p2)',
            'wo  = (p3 + yan)/(yan + 1)',
            'sil = (p4 + yan)/(yan + 1)',
            'q   = p0/(1 + yan)',
            'xc  = p5/(p5 + p6)',
            'fsp =  (p1 + p2)/(yan + 1)',
            'ol =  (p5 + p6)/(yan + 1)']
    inout = {'L':L}
    cv_mb.update(inout)

    hb = [  't  = p10',
            'f  = p8',
            'z  = p3 + f',
            'a  = p2 + p9 ',
            'k  = p9/(p2 + p9)',
            'c  = a + p0 + p1 + t',
            'y  = -a/2 + c - f - p0 - t + z',
            'xw = (5*c + 5*p4 - 2*p5 + p6 + 5*z - 5)/(2*c + 2*f + 2*t + 2*y + 2*z - 7)',
            'q1 = -2*c*xw/5 + 2*c - 2*f*xw/5 + 2*p4 - 2*p5/5 + 6*p6/5 - 2*t*xw/5 - 2*xw*y/5 - 2*xw*z/5 + 12*xw/5 + 2*z - 2',
            'q2 = (-2*c*xw + 5*c + 5*p4 + 3*p6 - 2*xw*z + 5*xw + 5*z - 5)/(2*(f + t + y - 1))']
    inout = {'hb':hb}
    cv_mb.update(inout)

    aug = [ 'qal = p5',
            'y   = p6 + qal',
            'f   = p4',
            'w   = p3 + f',
            'z   = p0 + y',
            'xc  = (2*p1 + p7 + 2*w + 2*z - 2)/(2*w + y + z - 2)',
            'qm  = 2*(2*p1*w + 2*p1*y - 2*p1 + 2*p2*w + p2*y + p2*z - 2*p2 + p7*w + p7*y - p7 + 2*w**2 + 2*w*y + 2*w*z - 4*w + 2*y*z - 2*y - 2*z + 2)/(2*w**2 + w*y + 3*w*z - 4*w + y*z - y + z**2 - 3*z + 2)']
    inout = {'aug':aug}
    cv_mb.update(inout)

    dio = [ 'qal = p6/2',
            'qn  = p4/2',
            'w   = p0 + p3 + 2*qal + qn',
            'qm  = (p2*qn + p5*qn/2 + p5*w/2 - p5/2)/(qn*w - qn + w**2 - 2*w + 1)',
            'xc  = (p5/2 - qm*qn - qm*w + qm)/qn',
            'f   = (-p0 - qal - qn + w)/w']
    inout = {'dio':dio}
    cv_mb.update(inout)

    opx = [ 'y = p3',
            'f = p4',
            'c = p5',
            'z = (c + f + p0 - p1 + y - 1)/(c + f + y - 2)',
            'q = (-c**2 - c*p0 + c*p1 - c*p2 + c + f**2 + f*p0 - f*p1 - f*p2 + 2*f*y - f + p0*y - p1*y - p2*y + 2*p2 + y**2 - y)/(c**2 + c*f + c*y - 3*c - f - y + 2)']
    inout = {'opx':opx}
    cv_mb.update(inout)

    g = [   'f = p3',
            'z = p2',
            'xc = p1/(1-z)']
    inout = {'g':g}
    cv_mb.update(inout)

    ol = [   'xc = p1']
    inout = {'ol':ol}
    cv_mb.update(inout)

    pl4tr = [   'k = p2',
                'ca = p1']
    inout = {'pl4tr':pl4tr}
    cv_mb.update(inout)

    abc = [   'ca = p1']
    inout = {'abc':abc}
    cv_mb.update(inout)

    k4tr = [    'na = p0',
                'ca = p1']
    inout = {'k4tr':k4tr}
    cv_mb.update(inout)

    ksp = [     'na = p1',
                'ca = p2']
    inout = {'ksp':ksp}
    cv_mb.update(inout)

    plc = [     'k  = p2',
                'ca = p1']
    inout = {'plc':plc}
    cv_mb.update(inout)

    pli = [     'k  = p2',
                'ca = p1']
    inout = {'pli':pli}
    cv_mb.update(inout)

    sp = [     'z  = p3',
                'y = 1 - p2 - z',
                'x = (-p1 + z + 1)/(z + 1)']
    inout = {'sp':sp}
    cv_mb.update(inout)

    ilm = [     'Q  = p0',
                'x = p1 + Q']
    inout = {'ilm':ilm}
    cv_mb.update(inout)

    ilmm = [     'Q  = p0',
                'i = 1 - p2',
                'g  = p3']
    inout = {'ilmm':ilmm}
    cv_mb.update(inout)

    ep = [     'Q = p1/2',
                'f = p2 + Q']
    inout = {'ep':ep}
    cv_mb.update(inout)

    bi = [  'f = p5',
            't = p4',
            'y = p3',
            'x = (-3*p1 - p2)/(f + t + y - 3)',
            'Q = 3*f*x/2 - 3*f/2 + 3*p0/2 + 3*t*x/2 - 3*t/2 + 3*x*y/2 - 3*x/2 - 3*y/2 + 3/2']
    inout = {'bi':bi}
    cv_mb.update(inout)

    mu = [  'f = p5',
            'c = p4',
            'n = p3',
            'y = c + f + n + p0',
            'x = (p1 + y - 1)/(y - 1)']
    inout = {'mu':mu}
    cv_mb.update(inout)

    chl = [   'f = p6',
            'QAl = (p0+p3)/2',
              'y = p2 + QAl',
             'Q4 = (f**2 + 5*f*p0 + f*p1 - 3*f*p4 - 8*f*qal + 2*f*y - 2*f + 5*p0*qal + 5*p0*y - 5*p0 + p1*qal + p1*y - p1 - 4*p4*qal - 2*p4*y - 2*p4 - 9*qal**2 - 8*qal*y + 8*qal + y**2 - 2*y + 1)/(f**2 + f*qal + 3*f*y - 7*f + 2*qal*y - 6*qal + 2*y**2 - 8*y + 6)',
              'x = (f*q4 - p4 + q4*qal + q4*y - q4)/(f + qal + y - 1)',
             'q1 = (f*q4 - 4*f*x/5 - 4*p5/5 + q4*qal + q4*y - q4 - 8*x*y/5 + 4*x/5)/(qal - y + 1)']
    inout = {'chl':chl}
    cv_mb.update(inout)
    
    return cv_mb

In [3]:
dirpath   = 'input_database_TC/'
db        = 'ig'

if db    == 'mb':
    dbase = 'metabasite_set_full_descriptions_NewForm.txt'
    cv_mb = get_cv_mb()
elif db  == 'mp':  
    dbase = 'metapelite_set_full_descriptions_NewForm.txt'
elif db  == 'ig':
    dbase = 'igG23_H18w_MAGEMin_descriptions.txt'
elif db == 'igd':
    dbase = 'igG23_THw_MAGEMin_descriptions.txt'
    
database  = dirpath+dbase

In [23]:
# replace functions
#-------------------#
def print_em_gb(em,din):
    dout = str(din)
    for i in din.free_symbols:
        if i in em:
            if (str(i)[0:2] == 'd_'):
                sym = str(i)[2:]+'_di.gb'
                dout= replaceSymbols(dout,str(i),sym)
            elif (str(i)[0:2] == 'o_'):
                sym = str(i)[2:]+'_or.gb'
                dout= replaceSymbols(dout,str(i),sym)
            elif (str(i)[0:2] == 'e_'):
                sym = str(i)[2:]+'_eq.gb'
                dout= replaceSymbols(dout,str(i),sym)
            else:
                sym = str(i)+'_eq.gb'
                dout= replaceSymbols(dout,str(i),sym)
        else:
                sym = 'z_b.'+str(i)
                dout= replaceSymbols(dout,str(i),sym)
            
    return dout
 
def print_em_shearM(em,din):
    din_dependent = sympify('0');
    for i in din.free_symbols: 
        if i in em:
            r = din.as_independent(i)
    #             print(sympify(r[1]))
            din_dependent += (sympify(r[1]))

    dout = str(din_dependent)
    for i in din.free_symbols:
        if i in em:
            if (str(i)[0:2] == 'd_'):
                sym = str(i)[2:]+'_di.ElShearMod'
                dout= replaceSymbols(dout,str(i),sym)
            elif (str(i)[0:2] == 'o_'):
                sym = str(i)[2:]+'_or.ElShearMod'
                dout= replaceSymbols(dout,str(i),sym)
            elif (str(i)[0:2] == 'e_'):
                sym = str(i)[2:]+'_eq.ElShearMod'
                dout= replaceSymbols(dout,str(i),sym)
            else:
                sym = str(i)+'_eq.ElShearMod'
                dout= replaceSymbols(dout,str(i),sym)

    return dout

def print_em_C(em,din):
    din_dependent = sympify('0');
    for i in din.free_symbols: 
        if i in em:
            r = din.as_independent(i)
            din_dependent += (sympify(r[1]))

    dout = str(din_dependent)
    for i in din.free_symbols:
        if i in em:
            if (str(i)[0:2] == 'd_'):
                sym = str(i)[2:]+'_di.C[i]'
                dout= replaceSymbols(dout,str(i),sym)
            elif (str(i)[0:2] == 'o_'):
                sym = str(i)[2:]+'_or.C[i]'
                dout= replaceSymbols(dout,str(i),sym)
            elif (str(i)[0:2] == 'e_'):
                sym = str(i)[2:]+'_eq.C[i]'
                dout= replaceSymbols(dout,str(i),sym)
            else:
                sym = str(i)+'_eq.C[i]'
                dout= replaceSymbols(dout,str(i),sym)
    return dout

def replaceSymbols(op,in_var,out_var):    
    regin  = 'r\'([' + in_var   +']*)\''
    regout = 'r\''  + out_var  +'\''
    t0 =  re.sub(in_var, out_var,op )  
    return(t0)
   
def eval_sf(x,sf):
    sf_ok = 1
    sf    = eval(sf)

    for  i in range(0,len(sf)):
        if sf[i] > -1e-10:
            sf_ok = 0
            break;
    
    return sf_ok


# get data from python dictionary functions
#-------------------------------------------#
def get_dpdx(ss,ph):
    #get entries
    pid = list(ss[ph]['p'].keys())
    cv  = list(ss[ph]['cv'].keys())

    # create array of variable symbols
    x_array = ''
    for i in range(0,len(cv)):
        x_array += 'x['+str(i)+'] '  
    x_array = symbols(x_array.strip())


    # differentiate and replace
    dp_dx = [[0] * (len(cv)) for i in range(len(pid))]
    for c,i in enumerate(pid):
        term = sympify(ss[ph]['p'][i])
        for d,j in enumerate(cv):
            dp_dx[c][d] = diff(term,cv[d])
            if (len(cv) == 1):
                dp_dx[c][d] = dp_dx[c][d].subs(cv[0],x_array)
            else:
                for k in range(0,len(cv)):
                    dp_dx[c][d] = dp_dx[c][d].subs(cv[k],x_array[k])
                    
    # turn integer to float for the C version
    for j,b in enumerate(dp_dx):
        for k,c in enumerate(b):
            tmp = dp_dx[j][k]
            for a in preorder_traversal(c):
                if isinstance(a, Rational):
                    tmp = tmp.subs(a, float(a))
            dp_dx[j][k] = tmp

    return dp_dx

def get_dsf_dx(ss,ph):

    sfid = list(ss[ph]['sf'].keys())
    cv  = list(ss[ph]['cv'].keys())
    
    # create array of variable symbols
    x_array = ''
    for i in range(0,len(cv)):
        x_array += 'x['+str(i)+'] '  
    x_array = symbols(x_array.strip())

    # differentiate and replace
    dsf_dx = [[0] * (len(cv)) for i in range(len(sfid))]
    sf     = [0] * (len(sfid))
    for c,i in enumerate(sfid):
        term = sympify(ss[ph]['sf'][i])
        sf[c] = -term;
        if (len(cv) == 1):
            sf[c] = sf[c].subs(cv[0],x_array)
        else:
            for k in range(0,len(cv)):
                sf[c] = sf[c].subs(cv[k],x_array[k])
        
        for d,j in enumerate(cv):
            dsf_dx[c][d] = - diff(term,cv[d])
            if (len(cv) == 1):
                dsf_dx[c][d] = dsf_dx[c][d].subs(cv[0],x_array)
            else:
                for k in range(0,len(cv)):
                    dsf_dx[c][d] = dsf_dx[c][d].subs(cv[k],x_array[k])
                    
                    
    # turn integer to float for the C version
    for j,b in enumerate(dsf_dx):
        for k,c in enumerate(b):
            tmp = dsf_dx[j][k]
            for a in preorder_traversal(c):
                if isinstance(a, Rational):
                    tmp = tmp.subs(a, float(a))
            dsf_dx[j][k] = tmp

    for k,c in enumerate(sf):
        tmp = sf[k]
        for a in preorder_traversal(c):
            if isinstance(a, Rational):
                tmp = tmp.subs(a, float(a))
        sf[k] = tmp

    return dsf_dx,sf

def get_idm(ss,ph):

    idmid = list(ss[ph]['idm'].keys())
    sf  = list(ss[ph]['sf'].keys())
    
    # create array of variable symbols
    x_array = ''
    for i in range(0,len(sf)):
        x_array += 'sf['+str(i)+'] '  
    x_array = symbols(x_array.strip())

    # differentiate and replace
    idm     = [0] * (len(idmid))
    for c,i in enumerate(idmid):
        term = sympify(ss[ph]['idm'][i])
        idm[c] = term;
        if (len(sf) == 1):
            idm[c] = idm[c].subs(sf[0],x_array)
        else:
            for k in range(0,len(sf)):
                idm[c] = idm[c].subs(sf[k],x_array[k])

    # turn integer to floar for the C version
    for k,c in enumerate(idm):
        tmp = idm[k]
        for a in preorder_traversal(c):
            if isinstance(a, Rational):
                tmp = tmp.subs(a, float(a))
        idm[k] = tmp

                
    return idm

def get_p(ss,ph):

    pid = list(ss[ph]['p'].keys())
    cv  = list(ss[ph]['cv'].keys())
    
    # create array of variable symbols
    x_array = ''
    for i in range(0,len(cv)):
        x_array += 'x['+str(i)+'] '  
    x_array = symbols(x_array.strip())

    # deplace
    p     = [0] * (len(pid))
    for c,i in enumerate(pid):
        term = sympify(ss[ph]['p'][i])
        p[c] = term;
        if (len(cv) == 1):
            p[c] = p[c].subs(cv[0],x_array)
        else:
            for k in range(0,len(cv)):
                p[c] = p[c].subs(cv[k],x_array[k])
                
    # turn integer to float for the C version    
    for k,c in enumerate(p):
        tmp = p[k]
        for a in preorder_traversal(c):
            if isinstance(a, Rational):
                tmp = tmp.subs(a, float(a))
        p[k] = tmp
        
    return p


# print functions
#-----------------#
def toMAGEMin_mem_size(ss,db,ph,op):

    os = '    '
    op += '/**\n'
    op += os+'allocate memory for '+ph+'\n'
    op += '*/\n'
    op += 'SS_ref G_SS_'+db+'_'+ph+'_init_function(SS_ref SS_ref_db, int EM_database, global_variable gv){\n'
    op += os+'\n'
    op += os+'SS_ref_db.is_liq    = '+'0'+';\n' 
    op += os+'SS_ref_db.symmetry  = '+str(ss[ph]['symmetry'])+';\n' 
    op += os+'SS_ref_db.n_sf      = '+str(len(ss[ph]['sf']))+';\n' 
    op += os+'SS_ref_db.n_em      = '+str(len(ss[ph]['p']))+';\n' 
    if ss[ph]['symmetry'] == 0:
        op += os+'SS_ref_db.n_v       = '+str(len(ss[ph]['v']))+';\n' 
    op += os+'SS_ref_db.n_w       = '+str(len(ss[ph]['W']))+';\n' 
    op += os+'SS_ref_db.n_xeos    = '+str(len(ss[ph]['p'])-1)+';\n' 
    op += os+'SS_ref_db.CstFactor = 0;\n' 
    op += os+'\n'
    op += os+'return SS_ref_db;\n' 
    op += '}\n'  
    op += '\n'  
    return(op)

def toMAGEMin_dpdx(ss,db,ph,dp_dx,op):
    os = '    '
    op += '\n' 
    op += '/**\n'
    op += os + 'Update dpdx matrix of '+ph+'\n'
    op += '*/\n'
    op += 'void dpdx_'+db+'_'+ph+'(void *SS_ref_db, const double *x){\n'
    op += os+'SS_ref *d  = (SS_ref *) SS_ref_db;\n'
    op += os+'double **dp_dx = d->dp_dx;\n\n'

    for  i in range(0,len(dp_dx)):
        for  j in range(0,len(dp_dx[0])):
            op += os+'dp_dx['+str(i)+']['+str(j)+'] = '+(str(dp_dx[i][j]))+';  '  
        op += os+'\n'
    op += '}\n'
    op += '\n'   

    return op

def toMAGEMin_ineq(ss,db,ph,dsf_dx,sf,op):
    out_name    = 'MAGEMin_Ineq_'+db+'_'+ph+".txt"
    os = '    '
    op += '/**\n'
    op += os + 'Inequality constraints for '+ph+'\n'
    op += '*/\n'
    op += 'void '+ph+'_c(unsigned m, double *result, unsigned n, const double *x, double *grad, void *data){\n'
    for i in range(0,len(sf)):
#         op += os+'double c'+ str(ii)+'(unsigned n, const double *x, double *grad, void *data){\n'   
        op += os+'result['+str(i)+'] = ( eps_sf + ' + str(sf[i])+');\n' 
    op += '\n' 
    op += os+'if (grad) {\n' 
    n = 0;
    for i in range(0,len(dsf_dx)):
        for j in range(0,len(dsf_dx[0])):
            op += os+os+'grad['+str(n)+'] = ' + str(dsf_dx[i][j]) + ';\n' 
            n += 1
    op += os+'}\n' 
    
    op += '\n'  
    op += os+'return;\n'    
    op += '};\n' 
    op += '\n'  
    
    return op

def toMAGEMin_p(ss,db,ph,p,op):
    os = '    '
    op += os+'\n' 
    op += '/**\n'
    op += os + 'Endmember fraction of '+ph+'\n'
    op += '*/\n'
    op += 'void px_'+db+'_'+ph+'(void *SS_ref_db, const double *x){\n'
    op += os+'SS_ref *d  = (SS_ref *) SS_ref_db;\n'
    op += os+'double *p = d->p;\n'
    
    for  i in range(0,len(p)):
        op += os+'    p['+str(i)+']           = '+(str(p[i]))+';\n'    
    op += '}\n'
    op += '\n'   

    return op

def toMAGEMin_obj(ss,db,ph,sf,idm,op):
    os = '    '
    op += os+'\n' 
    op += '/**\n'
    op += os + 'Objective function of '+ph+'\n'
    op += '*/\n'
    op += 'double obj_'+db+'_'+ph+'(unsigned n, const double *x, double *grad, void *SS_ref_db){\n'
    op += os+'SS_ref *d         = (SS_ref *) SS_ref_db;\n\n'
    op += os+'int n_em          = d->n_em;\n'
    op += os+'double P          = d->P;\n'
    op += os+'double T          = d->T;\n'
    op += os+'double R          = d->R;\n\n'

    op += os+'double *gb        = d->gb_lvl;\n'
    if (ss[ph]['symmetry']  == 0):
        op += os+'double *mat_phi   = d->mat_phi;\n'
    op += os+'double *mu_Gex    = d->mu_Gex;\n'
    op += os+'double *sf        = d->sf;\n'
    op += os+'double *mu        = d->mu;\n'

    op += os+'px_'+db+'_'+ph+'(SS_ref_db,x);\n\n'
    
    if (ss[ph]['symmetry']  == 1):
        op += os+'for (int i = 0; i < n_em; i++){\n'
        op += os+os+'mu_Gex[i] = 0.0;\n'
        op += os+os+'int it    = 0;\n'
        op += os+os+'for (int j = 0; j < d->n_xeos; j++){\n'
        op += os+os+os+'for (int k = j+1; k < n_em; k++){\n'
        op += os+os+os+os+'mu_Gex[i] -= (d->eye[i][j] - d->p[j])*(d->eye[i][k] - d->p[k])*(d->W[it]);\n'  
        op += os+os+os+os+'it += 1;\n'  
        op += os+os+os+'}\n'  
        op += os+os+'}\n'   
        op += os+'}\n'
        
    if (ss[ph]['symmetry']  == 0):
        op += os+'d->sum_v = 0.0;\n'
        op += os+'for (int i = 0; i < n_em; i++){\n'     
        op += os+os+'d->sum_v += d->p[i]*d->v[i];\n'     
        op += os+'}\n'
        op += os+'for (int i = 0; i < n_em; i++){\n'
        op += os+os+'d->mat_phi[i] = (d->p[i]*d->v[i])/d->sum_v;\n'
        op += os+'}\n'
        op += os+'\n'
        op += os+'for (int i = 0; i < d->n_em; i++){\n' 
        op += os+os+'mu_Gex[i] = 0.0;\n'
        op += os+os+'int it = 0;\n'
        op += os+os+'for (int j = 0; j < d->n_xeos; j++){\n'
        op += os+os+os+'for (int k = j+1; k < d->n_em; k++){\n'    
        op += os+os+os+os+'mu_Gex[i] -= (d->eye[i][j] - d->mat_phi[j])*(d->eye[i][k] - d->mat_phi[k])*(d->W[it]*2.0*d->v[i]/(d->v[j]+d->v[k]));\n'    
        op += os+os+os+os+'it += 1;\n'
        op += os+os+os+'}\n'
        op += os+os+'}\n'
        op += os+'}\n'
        
    op += os+'\n'
    for i in range(0,len(sf)):
        op += os+'sf['+str(i)+']          = '+(str(-sf[i]))+';\n'    
    op += os+'\n'
    op += os+'\n'
    for i in range(0,len(idm)):
        op += os+'mu['+str(i)+']          = gb['+str(i)+'] + R*T*creal(clog('+(str(ccode(idm[i]))).replace("pow", "cpow")+')) + mu_Gex['+str(i)+'];\n'
         
    op += os+'\n'

    op += os+'d->sum_apep = 0.0;\n'
    op += os+'for (int i = 0; i < n_em; i++){\n'
    op += os+os+'d->sum_apep += d->ape[i]*d->p[i];\n'
    op += os+'}\n'
    op += os+'d->factor = d->fbc/d->sum_apep;\n\n'

    op += os+'d->df_raw = 0.0;\n'
    op += os+'for (int i = 0; i < n_em; i++){\n'
    op += os+os+'d->df_raw += mu[i]*d->p[i];\n'
    op += os+'}\n'
    op += os+'d->df = d->df_raw * d->factor;\n\n'

    op += os+'if (grad){\n'
    op += os+os+'double *dfx    = d->dfx;\n'
    op += os+os+'double **dp_dx = d->dp_dx;\n'
    op += os+os+'dpdx_'+db+'_'+ph+'(SS_ref_db,x);\n'
    op += os+os+'for (int i = 0; i < (d->n_xeos); i++){\n'
    op += os+os+os+'dfx[i] = 0.0;\n'
    op += os+os+os+'for (int j = 0; j < n_em; j++){\n'
    op += os+os+os+os+'dfx[i] += (mu[j] - (d->ape[j]/d->sum_apep)*d->df_raw)*d->factor*dp_dx[j][i];\n'
    op += os+os+os+'}\n'
    op += os+os+os+'grad[i] = creal(dfx[i]);\n'
    op += os+os+'}\n'
    op += os+'}\n\n'
    op += os+'return d->df;\n' 
    op += '}\n'
    
    return op

def toMAGEMin_fct_collect(ss,db,op): #this generate the collector functions for various parts of the C code
    op += 'PC FUNCTION\n\n'    
    op += '   if (gv.EM_database == 0){\n'
    for i,ph in enumerate(ss):
        if i == 0:
            op += '      if (strcmp( name, "'+str(ph)+'") == 0 ){\n'
        else:
            op += '      else if (strcmp( name, "'+str(ph)+'") == 0 ){\n'
        op += '         G0 = obj_'+db+'_'+str(ph)+'(SS_ref_db.n_xeos, SS_ref_db.iguess, SS_ref_db.dfx, &SS_ref_db);\n'
        op += '      }\n'
    op += '      else{\n'
    op += '         printf("\\nsolid solution \'%s\' is not in the database\\n",name);\n'
    op += '      }\n'
    op += '   }\n'
    
    op += '\n\n'
    op += 'P2X\n\n'  
    
    
    op += '   if (gv.EM_database == 0){\n'
    for i,ph in enumerate(ss):
        if i == 0:
             op += '      if (strcmp( name, "'+str(ph)+'") == 0 ){\n'
        else:
            op += '      else if (strcmp( name, "'+str(ph)+'") == 0 ){\n'
        op += '         p2x_'+db+'_'+str(ph)+'(&SS_ref_db, eps);\n'
        op += '      }\n'
    op += '      else{\n'
    op += '         printf("\\nsolid solution \'%s\' is not in the database\\n",name);\n'
    op += '      }\n'
    op += '   }\n'
    
    op += '\n\n'
    op += 'NLOPT\n\n'  
    
    op += '   if (gv.EM_database == 0){\n'
    for i,ph in enumerate(ss):
        if i == 0:
             op += '      if (strcmp( gv.SS_list[index], "'+str(ph)+'")  == 0){\n'
        else:
            op += '      else if (strcmp( gv.SS_list[index], "'+str(ph)+'")  == 0){\n'
        op += '         SS_ref_db  = NLopt_opt_'+db+'_'+str(ph)+'_function( gv, SS_ref_db);}\n'
    op += '      else{\n'
    op += '         printf("\\nsolid solution \'%s index %d\' is not in the database\\n",gv.SS_list[index], index);\n'
    op += '      }\n'
    op += '   }\n'
    
    op += '\n\n'
    op += 'GSS INIT\n\n'  
    
    op += '   if (EM_database == 0){\n'
    for i,ph in enumerate(ss):
        if i == 0:
             op += '      if (strcmp( name, "'+str(ph)+'") == 0 ){\n'
        else:
            op += '      else if (strcmp( name, "'+str(ph)+'") == 0 ){\n'
        op += '         SS_ref_db  = G_SS_'+db+'_'+str(ph)+'_init_function(SS_ref_db, EM_database, gv); }\n'
    op += '      else{\n'
    op += '         printf("\\nsolid solution \'%s\' is not in the database\\n",name);\n'
    op += '      }\n'
    op += '   }\n'
         
    op += '\n\n'
    op += 'GSS FUNCTION\n\n' 
    op += '   /* Associate the right solid-solution data */\n'
    op += '   for (int FD = 0; FD < gv.n_Diff; FD++){				/* cycle twice in order to get gb_P_eps to calculate densities later on */\n'
    op += '      if (FD == 8 || FD == 9){				// dG/dP0 to get Volume at P = 1bar\n'
    op += '         SS_ref_db.P = 1.+ gv.gb_P_eps*gv.pdev[0][FD];\n'
    op += '         SS_ref_db.T = T + gv.gb_T_eps*gv.pdev[1][FD];\n'
    op += '      }\n'
    op += '      else{\n'
    op += '         SS_ref_db.P = P + gv.gb_P_eps*gv.pdev[0][FD];\n'  
    op += '         SS_ref_db.T = T + gv.gb_T_eps*gv.pdev[1][FD];\n'  
    op += '      }\n'  

    for i,ph in enumerate(ss):
        if i == 0:
             op += '      if (strcmp( name, "'+str(ph)+'") == 0 ){\n'
        else:
            op += '      else if (strcmp( name, "'+str(ph)+'") == 0 ){\n'
            
        #this part has to be included manually, or using the endmembers composition
#         op += '         if (z_b.bulk_rock[10] == 0.){\n'
#         op += '            SS_ref_db.ss_flags[0]  = 0;\n'    
#         op += '         }\n'        

        op += '         SS_ref_db  = G_SS_'+db+'_'+str(ph)+'_function(SS_ref_db, EM_database, gv.len_ox, z_b, eps);	}\n'
    op += '      else{\n'
    op += '         printf("\\nsolid solution \'%s\' is not in the database\\n",name);	}\n'
    op += '\n'
    op += '      for (int j = 0; j < SS_ref_db.n_em; j++){\n'
    op += '         SS_ref_db.mu_array[FD][j] = SS_ref_db.gbase[j];\n'
    op += '         // printf(" %+10.10f",SS_ref_db.gbase[j]);\n'
    op += '      }\n'
    op += '   }'


    return op

def toMAGEMin_make(ss,db,ph,op):
    
    W = ss[ph]['W']
    
    if ss[ph]['symmetry'] == 0:
        v  = ss[ph]['v']
    
    n_em    = len(ss[ph]['p'])
    make    = [0]*n_em;
     
    emname = list(ss[ph]['make'].keys())
    for c,i in enumerate(emname):
        ss[ph]['make'][i] = replaceSymbols(ss[ph]['make'][i],'per','pertmp')
    
    for c,i in enumerate(emname):
        make[c] = sympify(ss[ph]['make'][i])
        
    exclude = [sympify('P'),sympify('T')];
    em = [];
    for i in make:
        for key in i.free_symbols:
            if key not in em and key not in exclude:
                em.append(key)
     
    os = '    '
    op += '/**\n'
    op += '   retrieve reference thermodynamic data for '+db+'_'+ph+'\n'
    op += '*/\n'
    op += 'SS_ref G_SS_'+db+'_'+ph+'_function(SS_ref SS_ref_db, int EM_database, int len_ox, bulk_info z_b, double eps){\n'
    op += os+'\n'
    op += os+'int i, j;\n'
    op += os+'int n_em = SS_ref_db.n_em;\n'
    op += os+'\n'
    op += os+'char   *EM_tmp[] 		= {'
    for i,val in enumerate(emname):
        if i < n_em-1:
            op += '"'+str(val)+'",'
        else:
            op += '"'+str(val)+'"'
    op += '};\n'
    op += os+'for (int i = 0; i < SS_ref_db.n_em; i++){\n'
    op += os+os+'strcpy(SS_ref_db.EM_list[i],EM_tmp[i]);\n'
    op += os+'};\n'

    op += os+'\n'
    for i in range(0,len(ss[ph]['W'])):
        rhs = re.sub(r'([PT])', r'SS_ref_db.\1', W[i])
        op += os+'SS_ref_db.W['+str(i)+'] = '+rhs+';\n'
    op += os+'\n'
    if ss[ph]['symmetry'] == 0:
        for i in range(0,len(ss[ph]['v'])):
            op += os+'SS_ref_db.v['+str(i)+'] = '+str(v[i])+';\n'
        op += os+'\n'
    op += os+'\n'
    for i in em:
        if (str(i)[0:2] == 'd_'):
            opt = 'disordered'
            suf = 'di';
            name = str(i)[2:] 
        elif (str(i)[0:2] == 'o_'):
            opt = 'ordered'
            suf = 'or';
            name = str(i)[2:] 
        elif (str(i)[0:2] == 'e_'):
            opt = 'equilibrium'
            suf = 'eq';
            name = str(i)[2:] 
        else:
            opt = 'equilibrium'
            suf = 'eq';
            name = str(i)

        op += os+'em_data '+name+'_'+suf+' 		= get_em_data(		EM_database, \n'
        op += os+'										len_ox,\n'
        op += os+'										z_b,\n'
        op += os+'										SS_ref_db.P,\n'
        op += os+'										SS_ref_db.T,\n'
        op += os+'										"'+name+'", \n'
        op += os+'										"'+opt+'"	);\n'
        op += os+'\n'

    for i in range(0,n_em):
        op += os+ 'SS_ref_db.gbase['+str(i)+'] 		= '+print_em_gb(em,make[i])+';\n'
    op += os+'\n'

    for i in range(0,n_em):
        op += os+ 'SS_ref_db.ElShearMod['+str(i)+'] 	= '+print_em_shearM(em,make[i])+';\n'
    op += os+'\n'

    op += os+'for (i = 0; i < len_ox; i++){\n'
    for i in range(0,n_em):
        op += os+os+ 'SS_ref_db.Comp['+str(i)+'][i] 	= '+print_em_C(em,make[i])+';\n'
    op += os+'}\n'
    op += os+'\n'
    op += os+'for (i = 0; i < n_em; i++){\n'
    op += os+os+'SS_ref_db.z_em[i] = 1.0;\n'
    op += os+'};\n'
    op += os+'\n'
    op += os+'!!! RANGE MISSING, WAIT FOR UPDATE !!!\n\n'
#     for i in range(0,n_em-1):
#         op += os+ 'SS_ref_db.bounds_ref['+str(i)+'][0] = '+str(float(Range[i][0]))+'+eps;  SS_ref_db.bounds_ref['+str(i)+'][1] = '+str(float(Range[i][1]))+'-eps;\n'
#     op += os+'\n'
    op += os+'return SS_ref_db;\n'
    op += '}\n'
    op += '\n'
    
    # there is a strange function conflict with per...
    # here per is first replaced with pertmp then replaced back to per
    op = replaceSymbols(op,'pertmp','per')
    
    
    return op 

def toMAGEMin_gen_PC(ss,db,ph,stp,sf,op):


#     sf2   = str(sf)
    d     = len(ss[ph]['p']) - 1
    shift = 0.0001  
    print("Database               : [",db,"]")
    print("Solution phase         : [",ph,"]")
    print("Dims                   : [",d,"]")
    print("step length            : [",stp,"]") 
 
    # generalized nested loop
    xeos_list   = [];
    min_val     = 0;
    ip          = [min_val]*(d+1);
    nstep       = [min_val]*(d+1);
    min_arr     = [min_val]*(d+1);
    max_arr     = [min_val]*(d+1);

    for i in range(0,d):
        nstep[i+1]   = int((ss[ph]['range']['ub'][i] -ss[ph]['range']['lb'][i])/stp)+1;
        min_arr[i+1] = ss[ph]['range']['lb'][i];
        max_arr[i+1] = nstep[i+1];

    print("number of subdivisions :",nstep) 
    t = time.time()
    xeos_list   = [];
    pp          = d
    while ip[0] == min_val:

        xcv       = np.array(ip[1:d+1], dtype=np.float32)*stp+shift
        xcv      += np.array(min_arr[1:d+1], dtype=np.float32)
#         sf_ok   = eval_sf(x,sf2)
        sf_ok = pc_gen(xcv,sf)

        if sf_ok == 1:
            xeos_list.append(xcv)

        ip[pp] += 1;
        while ip[pp] == max_arr[pp]:
            ip[pp] = min_val;
            pp -= 1;
            if pp == 1:
                print('*')
            ip[pp] += 1;
            if (ip[pp] != nstep[pp]):
                pp = d    
                
    elapsed = time.time() - t
    out   = np.array(xeos_list)
    npc   = len(out);
    print("time to generate PC    :",elapsed, "seconds")
    print("number of generated PC :",npc)

    nxeos = d;
    
    op   += 'struct ss_pc '+db+'_'+ph+'_pc_xeos['+str(npc)+'] = {\n';
    for i in range(0,npc):
        op += '{{'
        for j in range(0,nxeos):
            op += str('%1.6f' %out[i][j])
            if j < nxeos-1:
                op += (',')
            else:
                if i < npc-1:
                    op += '}},'
                else:
                    op += '}}};'
        op += '\n'
    op += '\n'    
    return op    

def toMAGEMin_p2cv(ss,db,ph,cv_mb,op):
    
    in_var = list(ss[ph]['cv'].keys())
    n_em   = len(ss[ph]['p'])

    cv     = cv_mb[ph]
    pout   = [0]*(n_em-1)
    for ii, val in enumerate(cv):
        t0 = str(val)
        for i in range(n_em-1,-1,-1):
            inv  = 'p'+str(i)
            outv = 'd->p['+str(i)+']'
            t0   = replaceSymbols(t0,inv,outv) 

        for i in range(n_em-2,-1,-1):
            inv  = in_var[i]
            outv = 'd->iguess['+str(i)+']'
            t0   = replaceSymbols(t0,inv,outv) 

        pout[ii] = t0

    os = '    '
    op += '/**\n'
    op += os + 'Endmember to xeos for '+ph+'\n'
    op += '*/\n'
    op += 'void p2x_'+db+'_'+ph+'(void *SS_ref_db, double eps){\n'
    op += os+'SS_ref *d  = (SS_ref *) SS_ref_db;\n'
    op += os+'\n' 
    for i in pout:
        op += os+str(i)+';\n'

    op += os+'\n' 
    op += os+'for (int i = 0; i < d->n_xeos; i++){\n' 
    op += os+os+'if (d->iguess[i] < d->bounds[i][0]){\n' 
    op += os+os+os+'d->iguess[i] = d->bounds[i][0];\n' 
    op += os+os+'}\n' 
    op += os+os+'if (d->iguess[i] > d->bounds[i][1]){\n' 
    op += os+os+os+'d->iguess[i] = d->bounds[i][1];\n' 
    op += os+os+'}\n' 
    op += os+'}\n' 
    op += '}\n'
    op += '\n'
    return(op)
    
def save2file(db,ph,name,op):
    out_name    = 'toMAGEMin_'+name+'_'+db+'.txt'
    text_file   = open(out_name, "w")
    n           = text_file.write(op)
    print("saved to file   :",out_name, "...")
    
def get_pc_gen(db,ph):
    op = ''
    op += 'def pc_gen(xcv,sf):\n'
    op += '    sf_ok = 1\n';
    op += '\n'  
    cv = list(ss[ph]['cv'].keys())  
    for i,j in enumerate(cv):
        op += '    '+str(j)+' = xcv['+str(i)+'] \n';
    op += '\n'  
    for i,j in enumerate(ss[ph]['sf']):
        op += '    sf['+str(i)+'] = '+str(ss[ph]['sf'][j])+'\n';

    op += '\n'  
    op += '    for i in range(0,len(sf)):\n'
    op += '        if sf[i] <= 1e-10:\n'
    op += '            sf_ok = 0\n'
    op += '\n'  
    op += '    return sf_ok'  

    return op


In [24]:
# turn database into python dictionary
with open(database, 'r') as f2:
    data = f2.read()
    data = replaceSymbols(data,'h2o','wat')
        
line   = []
tmp_ss = []
ss     = {}
si     = 0

# split file per line
for c,i in enumerate(data):
    if i == '\n':
        se = c
        line.append(data[si:se])
        si = c+1
# get starting line of the ss description
for i in range(1,len(line)):
    if line[i] == '#' and line[i-1] == '#':
        start = i;
        break

# get solution phase starting and ending lines
for i in range(start,len(line)):   
    if line[i][0:3] == ' ==' and line[i-1] == '#':
        tmp_ss.append(i+1)

tmp_ss.append(len(line))       
        
# go through every phase and get the mane of the phase to store in a dictionary     
for i in range(0,len(tmp_ss)-1):  
    phase  = ''
    mk   = []
    for j in range(tmp_ss[i],tmp_ss[i+1]):
        if (line[j] == ' starting guesses'):
            sg = j
            ss_name = re.findall('\((.*?)\)', line[j+1])
            phase = ss_name[0]
            inout = {phase:[tmp_ss[i],tmp_ss[i+1]]}
            ss[phase] = inout

        if (line[j] == ' site fractions'):
            sf = j
        if (line[j][0:13] == ' non-ideality'): 
            ni = j
        if (line[j][0:13] == ' ideal mixing'): 
            im = j
        if (line[j][0:8] == ' "make" '): 
            mk = j
        if (line[j][0:12] == ' proportions'): 
            pr = j
                
    if not mk:
        mk = tmp_ss[i+1]-1

    # get margules
    for j in range(ni+1,mk-1):
        if len(line[j]) > 0:
            res = line[j][line[j].find("(")-1]
            if res == 'W':
                if not 'W' in ss[phase]:
                    inout = {'W':[]}
                    ss[phase].update(inout)
                
                marg = line[j].split("=")
                ss[phase]['W'].append(marg[1].strip())
                    
            if res == 'v':
                if not 'v' in ss[phase]:
                    inout = {'v':[]}
                    ss[phase].update(inout)
                
                marg = line[j].split("=")
                ss[phase]['v'].append(marg[1].strip())
    
    #site fraction
    for j in range(sf+1,pr-1):
        if len(line[j]) > 0:
            if not 'sf' in ss[phase]:
                inout = {'sf':{}}
                ss[phase].update(inout)
                
            sitef = line[j].split("=")
            lhs   = re.sub(r'\(|\)', '', sitef[0]).strip()
            rhs   = re.sub(r'([-+]\s*)?(\d+)([a-zA-Z]{1,3})', r'\g<1>\g<2>*\g<3>', sitef[1], flags=re.IGNORECASE).strip()
    
            inout = {lhs:rhs.lower()}
            ss[phase]['sf'].update(inout)
     
    # compositional variables
    for j in range(sg+1,sf-1):
        if len(line[j]) > 0:
            if not 'cv' in ss[phase]:
                inout = {'cv':{}}
                ss[phase].update(inout)
                
            cv = line[j].split("=")
            lhs = re.sub(r'\([^)]*\)', '', cv[0]).strip()
            inout = {lhs.lower():cv[1].strip()}
            ss[phase]['cv'].update(inout)
                     
    # proportions      
    for j in range(pr+1,im-1):
        if len(line[j]) > 0:
            if not 'p' in ss[phase]:
                inout = {'p':{}}
                ss[phase].update(inout)
                
            prop = line[j].split("=")
            rhs  = result = re.sub(r'([-+]\s*)?(\d+)([a-zA-Z]{1,3})', r'\g<1>\g<2>*\g<3>', prop[1], flags=re.IGNORECASE).strip()
            
            inout = {prop[0].strip():rhs.lower()}
            ss[phase]['p'].update(inout)   
            
    # range         
    if not 'range' in ss[phase]:
        inout = {'range':{}}
        ss[phase].update(inout)
        
        inout = {'lb':[0]*(len(ss[phase]['p'])-1)}
        ss[phase]['range'].update(inout)
        inout = {'ub':[1]*(len(ss[phase]['p'])-1)}
        ss[phase]['range'].update(inout)   
         
    cvid = 0;
    for j in range(sg+1,sf-1):
        if len(line[j]) > 0:

            cv = line[j].split("=")
            
            lb = range_number = re.search(r"range\s+([-+]?\d*\.\d+|[-+]?\d+)\s*<", str(cv)).group(1)
            ub = number_after_gt = re.search(r">\s*([-+]?\d*\.\d+|[-+]?\d+)", str(cv)).group(1)
            ss[phase]['range']['lb'][cvid] = float(lb);
            ss[phase]['range']['ub'][cvid] = float(ub);
            cvid += 1;
#             print(float(lb),ub)
            
            
    # ideal mixing              
    for j in range(im+1,ni-1):
        if len(line[j]) > 0:
            if not 'idm' in ss[phase]:
                inout = {'idm':{}}
                ss[phase].update(inout)
                
            idm = line[j].split("=")
            lhs = re.sub(r'\(|\)', '', idm[0]).strip()
            rhs =  re.sub(r"\(([^\(\)\+\-\*/]+)\)", lambda match: match.group(1), idm[1]).strip()

            inout = {lhs:rhs}
            ss[phase]['idm'].update(inout)    
            
            
    if mk == tmp_ss[i+1]-1: #then not composite make definition
        if not 'make' in ss[phase]:
            inout = {'make':{}}
            ss[phase].update(inout)
            
        for x in ss[phase]['p'].items():
            inout = {x[0]:x[0]}
            ss[phase]['make'].update(inout)   
    else:
        if not 'make' in ss[phase]:
            inout = {'make':{}}
            ss[phase].update(inout)
            
            
        for x in ss[phase]['p'].items():
            inout = {x[0]:x[0]}
            
            for j in range(mk+1,tmp_ss[i+1]-1):
                if len(line[j]) > 0:
                    make = line[j].split("=")
                    if x[0] == make[0].strip():
                        rhs = re.sub(r'\([^)]*\)', '', make[1]).strip()
                        rhs2 = re.sub(r'\b(\w+)-(\w+)\b', r'\1_\2', rhs)
                        rhs3 = re.sub(r'(\d+)\s+([a-zA-Z]+)', r'\1*\2', rhs2)
                        inout = {x[0]:rhs3}
 
            ss[phase]['make'].update(inout)   
            
    if 'v' in ss[phase]:
        inout = {'symmetry':0}
    else:
        inout = {'symmetry':1}
    ss[phase].update(inout) 
           
    # turn into float for C
    for j,b in enumerate(ss[phase]['W']):
        tmp = sympify(b)
        for a in preorder_traversal(tmp):
            if isinstance(a, Rational):
                tmp = tmp.subs(a, float(a))
        ss[phase]['W'][j] = str(tmp)

    if 'v' in ss[phase]:
        for j,b in enumerate(ss[phase]['v']):
            tmp = sympify(b)
            for a in preorder_traversal(tmp):
                if isinstance(a, Rational):
                    tmp = tmp.subs(a, float(a))
            ss[phase]['v'][j] = str(tmp)

In [25]:
op = ''
ph = ''
op = toMAGEMin_fct_collect(ss,db,op)
name = 'collect_fct'
save2file(db,ph,name,op)

saved to file   : toMAGEMin_collect_fct_ig.txt ...


In [26]:
### PRINT ALL MEM SIZE ###
op  = ''
for ph in ss:
    op        = toMAGEMin_mem_size(ss,db,ph,op)

name = 'mem_size'
save2file(db,ph,name,op)


saved to file   : toMAGEMin_mem_size_ig.txt ...


In [27]:
### PRINT ALL MAKE ###
op  = ''
for ph in ss:
    print(ph)
    op        = toMAGEMin_make(ss,db,ph,op)

name = 'make'
save2file(db,ph,name,op)

liq_G23Hw
fl_G23H
fsp_H21
spn_T21
g_G23
ol_H18
opx_G23
cpx_G23
ilm_W23
hb_G16
bi_G23
ep_H11
cd_H18
saved to file   : toMAGEMin_make_ig.txt ...


In [28]:
### PRINT ALL DPDX ###
op  = ''
for ph in ss:
    dp_dx     = get_dpdx(ss,ph)
    op        = toMAGEMin_dpdx(ss,db,ph,dp_dx,op)
# print(op)

name = 'dpdx'
save2file(db,ph,name,op)

saved to file   : toMAGEMin_dpdx_ig.txt ...


In [30]:
### PRINT ALL OBJ ###
op  = ''
for ph in ss:
    idm       = get_idm(ss,ph)
    dsf_dx,sf = get_dsf_dx(ss,ph)
    op        = toMAGEMin_obj(ss,db,ph,sf,idm,op)
# print(op)

name = 'obj'
save2file(db,ph,name,op)

saved to file   : toMAGEMin_obj_ig.txt ...


In [31]:
### PRINT ALL INEQUALITY CONSTRAINTS ###
op  = ''
for ph in ss:
    dsf_dx,sf = get_dsf_dx(ss,ph)
    op        = toMAGEMin_ineq(ss,db,ph,dsf_dx,sf,op)
# print(op)

# name = 'ineq'
save2file(db,ph,name,op)

saved to file   : toMAGEMin_obj_ig.txt ...


In [32]:
### PRINT ALL ENDMEMBER FRACTIONS ###
op  = ''
for ph in ss:
    p         = get_p(ss,ph)
    op        = toMAGEMin_p(ss,db,ph,p,op)
# print(op)

name = 'p'
save2file(db,ph,name,op)

saved to file   : toMAGEMin_p_ig.txt ...


In [69]:
### PRINT ALL PSEUDOCOMPOUNDS ###
op  = ''
stp = 0.33
# for ph in ss:
#     dsf_dx,sf = get_dsf_dx(ss,ph)
#     op        = toMAGEMin_gen_PC(ss,db,ph,stp,sf,op)
# # print(op)

# name = 'PC_list'
# save2file(db,ph,name,op)

# ph = 'liq_G23Hw'

# Here a function to check site-fraction is generated on the fly for performance reasons
# del pc_gen
ph = 'hb_G16'
pc = get_pc_gen(db,ph)
sf = [0]*len(ss[ph]['sf'])
exec(pc)

op   = toMAGEMin_gen_PC(ss,db,ph,stp,sf,op)
name = 'PC_list_'+ph
save2file(db,ph,name,op)


Database               : [ ig ]
Solution phase         : [ hb_G16 ]
Dims                   : [ 10 ]
step length            : [ 0.33 ]
number of subdivisions : [0, 4, 4, 4, 4, 4, 4, 4, 4, 7, 7]


KeyboardInterrupt: 

In [16]:
ss['liq_G23Hw']['sf']

{'pq': '1 - wo - sl - jd - fa - fo - hm - ek - ti - kj - wat + 1/4*yct*(4 - 3*ek - 3*fa - 3*fo - 3*hm - 3*jd - 3*kj - 3*sl - 3*ti - 3*wo - 3*wat)',
 'psl': 'sl + 3/4*yct*sl + (-yct)',
 'pwo': 'wo + 3/4*yct*wo + (-yct)',
 'pjd': 'jd + 3/4*yct*jd',
 'phm': 'hm + 3/4*yct*hm',
 'pek': 'ek + 3/4*yct*ek',
 'pti': 'ti + 3/4*yct*ti',
 'pkj': 'kj + 3/4*yct*kj',
 'pct': 'yct',
 'pol': 'fo + fa + 3/4*yct*(fo + fa)',
 'sumT': '1 - wat + (-3/4*yct)*wat',
 'mgM': '4*fo*(1 + 3/4*yct)',
 'feM': '4*fa*(1 + 3/4*yct)',
 'CaM': 'wo*(1 + 3/4*yct) + (-yct)',
 'AlM': 'sl*(1 + 3/4*yct) + (-yct)',
 'sumM': '(4*fo + 4*fa + sl + wo)*(1 + 3/4*yct) + (-2*yct)',
 'xh': 'wat*(1 + 3/4*yct)',
 'xv': '1 - wat + (-3/4*yct)*wat'}

In [18]:
ss['liq_G23Hw']['cv'].keys()

dict_keys(['wo', 'sl', 'fo', 'fa', 'jd', 'hm', 'ek', 'ti', 'kj', 'yct', 'wat'])